In [121]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/geeksd/training_set_features.csv
/kaggle/input/geeksd/test_set_features.csv
/kaggle/input/geeksd/training_set_labels.csv
/kaggle/input/geeksd/submission_format.csv


In [122]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:

# Load the features and labels
train_features = pd.read_csv('/kaggle/input/geeksd/training_set_features.csv')
train_labels = pd.read_csv('/kaggle/input/geeksd/training_set_labels.csv')

# Merge the features and labels on the common identifier if necessary
data = train_features.merge(train_labels, on='respondent_id')

# Display the first few rows of the data

data.head()

In [ ]:
data.describe()

In [ ]:
data.nunique()

In [ ]:
data.isnull().sum()

In [ ]:
data.info()

In [ ]:
sns.heatmap(data.isnull(), cmap='Blues', cbar=False, yticklabels=False, xticklabels=data.columns);

In [ ]:

  if data is None:
        raise ValueError("The DataFrame 'data' is None. Please check the file path and format.")
data.drop(['employment_occupation','employment_industry','health_insurance'], axis=1, inplace=True)
data.head()

In [ ]:
sns.heatmap(data.isnull(), cmap='Blues', cbar=False, yticklabels=False, xticklabels=data.columns);

In [ ]:
sum(data.isna().sum(axis=1) > 0)

In [ ]:

data.dtypes

In [ ]:
sns.heatmap(data.isnull(), cmap='Blues', cbar=False, yticklabels=False, xticklabels=data.columns);

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.preprocessing import LabelEncoder
df_simple_imputer = data.copy()
cat_cols = [col for col in data.columns if data[col].dtype == 'O']
cont_cols = [col for col in data.columns if col not in cat_cols]

numeric_cols = data.select_dtypes(include=['float64', 'int64']).columns
categorical_cols = data.select_dtypes(include=['object']).columns

imputer = SimpleImputer(strategy='mean')

df_simple_imputer[cont_cols] = imputer.fit_transform(data[cont_cols])
imputer = SimpleImputer(strategy='most_frequent')

df_simple_imputer[cat_cols] = imputer.fit_transform(data[cat_cols])


In [ ]:
sns.heatmap(df_simple_imputer.isnull(), cmap='Blues', cbar=False, yticklabels=False, xticklabels=data.columns);

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score, classification_report
X = df_simple_imputer.drop(columns=['xyz_vaccine','seasonal_vaccine'])
y = df_simple_imputer['xyz_vaccine']
z = df_simple_imputer['seasonal_vaccine']
# Separate numerical and categorical columns
numeric_cols = X.select_dtypes(include=['float64', 'int64']).columns
categorical_cols = X.select_dtypes(include=['object']).columns

# Define preprocessing steps for numeric and categorical data
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Combine preprocessing steps using ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])
# Create a pipeline combining preprocessing steps and logistic regression model
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LinearRegression())
])
model.fit(X, y)
model.fit(X, z)

In [ ]:
test = pd.read_csv('/kaggle/input/geeksd/test_set_features.csv')
test.head()

In [ ]:
 if test is None:
        raise ValueError("The DataFrame 'data' is None. Please check the file path and format.")
test.drop(['employment_occupation','employment_industry','health_insurance'], axis=1, inplace=True)
test.head()

In [ ]:
sns.heatmap(test.isnull(), cmap='Blues', cbar=False, yticklabels=False, xticklabels=test.columns);

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.preprocessing import LabelEncoder
simple_imputer = test.copy()
cat_cols = [col for col in test.columns if data[col].dtype == 'O']
cont_cols = [col for col in test.columns if col not in cat_cols]

numeric_cols = test.select_dtypes(include=['float64', 'int64']).columns
categorical_cols = test.select_dtypes(include=['object']).columns

imputer = SimpleImputer(strategy='mean')

simple_imputer[cont_cols] = imputer.fit_transform(test[cont_cols])
imputer = SimpleImputer(strategy='most_frequent')

simple_imputer[cat_cols] = imputer.fit_transform(test[cat_cols])


In [ ]:
sns.heatmap(simple_imputer.isnull(), cmap='Blues', cbar=False, yticklabels=False, xticklabels=simple_imputer.columns);

In [ ]:
y_pred = model.predict(simple_imputer)


In [ ]:
y_te = pd.read_csv('/kaggle/input/geeksd/submission_format.csv')
y_true = y_te.h1n1_vaccine
z_true = y_te.seasonal_vaccine

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score

# Assuming 'y_true' and 'y_pred' are predictions from a regression model
mse = mean_squared_error(y_true, y_pred)
r2 = r2_score(y_true, y_pred)
me = mean_squared_error(z_true, y_pred)
r = r2_score(z_true, y_pred)

print("Mean Squared Error: of xyz", mse)
print("R-squared:", r2)
print("Mean Squared Error: of seasonal", me)
print("R-squared:", r)
